# Setup

**Note to Mac users:**
* You might get this error: `Unable to revert mtime: /Library/Fonts`
* The solution is to install the *libmagic* brew: `brew install libmagic`
* Reference: [Python MacOS Error: Unable to revert mtime: /Library/Fonts](https://stackoverflow.com/questions/62279920/python-macos-error-unable-to-revert-mtime-library-fonts)

In [ ]:
from setup import *

## Set Variables

### Instructions

Gather the following variables:

* database name
* path to the Net-Manage repository
* private data directory
* path to store the output
* Meraki API key (if applicable)
* Credentials to login to devices (if applicable)

**Note:** The inventory file is statically set to *private_data_dir/inventory/hosts*

#### Explanation of Variables

### database name

defaults to the current date, in YYYY-MM-DD format--e.g., ```2023-01-05.db```

##### path to the Net-Manage repository

defaults to ```~/source/repos/InsightSSG/Net-Manage```

##### Path to the Ansible Private Data Directory

This is the folder that contains the inventory folder (not the host file).<br><br>
For example, If your folder structure looks like this:```tenants/client_a/inventory/hosts```<br><br>
Then the private data directory would be```tenants/client_a```

##### path to store the output

Path to store the database and other script output. Defaults to 'private_data_dir/output'

**note:** Ansible-runner output, like artifacts, will still be stored in the private data directory

##### Example folder structure

* Net-Manage path: *~/source/repos/InsightSSG/Net-Manage*

* Private data directory: *~/tenant_data/*

* Inventory file: *~/tenant_data/inventory/hosts*

### Enter Variables

In [ ]:
api_key, db, db_path, inventories, nm_path, out_path, private_data_dir = hp.set_vars()
play_path = f'{nm_path}/playbooks'
username, password = hp.get_creds()

# Select and Run Collectors

## Select the hostgroups to run the collectors on

**Note:**

* If running this in a miniconda environment and the widget does not display, run the command below.
* See: https://stackoverflow.com/questions/36351109/ipython-notebook-ipywidgets-does-not-show

```
jupyter nbextension enable --py --sys-prefix widgetsnbextension
```

In [ ]:
hostgroup_select = select_hostgroups(private_data_dir)
for key in sorted(hostgroup_select.keys()):
    print(key.upper())
    display(widgets.GridBox(hostgroup_select[key],
                            layout=widgets.Layout(grid_template_columns='repeat(3, 200px)')))
    print('\n')

## Select the collectors to run

In [ ]:
collector_select = select_collectors(collector_select, hostgroup_select)
for key in sorted(collector_select.keys()):
    print(key.upper())
    display(widgets.GridBox(collector_select[key],
                            layout=widgets.Layout(grid_template_columns='repeat(3, 250px)')))
    print('\n')

### Select Meraki Organizations and Networks (if applicable)

#### Instructions

This optional cell allows you to specify which orgs and/or networks to run collectors on.

**Important! (Please Read):**

* This is alpha code that has not yet been fully tested.
<br><br>
* I recommend skipping it unless you need to get the results for *only* specific orgs and/or networks.
  * See the explanation below for examples of when you might need to execute this cell.
<br><br>
* If you execute this cell by accident, just leave the input values blank.

**Explanation:**

* By default (if you do not execute this cell) the 'meraki_get_org' collectors will query all orgs that have API access enabled.
  * Executing this cell gives you the option to specify one or more orgs
  * If you execute this cell and do not specify any orgs, it will default back to querying all orgs
<br><br>
* By default (if you do not execute this cell) the 'meraki_get_network' functions will query all network IDs for the orgs you specify
  * If you do not specify any orgs or any network IDs, then all network IDs for all orgs will be queried
<br><br>
* Most of the time, querying all orgs and all network IDs is not a problem, since the data can be filtered later
  * However, this code was added for those few people who have tens of thousands of networks (or more)

**Note:**

* Currently there is a bug that can cause the collectors to fail when the following conditions are met:
    1. You use an API key to run collectors on orgs that API key has access to
    1. You use the same database to run collectors on orgs that a *different* API key has access to
    1. This is on my ToDo list to fix. For now, the workaround is to use a separate database.

#### Enter Variables

In [ ]:
import helpers as hp
import run_collectors as rc
from collectors import collectors as cl
# from collectors import meraki as mc
import meraki as mc

importlib.reload(cl)
importlib.reload(hp)
importlib.reload(mc)
importlib.reload(rc)

networks, orgs = hp.get_user_meraki_input()

## Run Collectors

In [ ]:
%%time

# Reload imported modules (used for debugging)
import helpers as hp
import run_collectors as rc
from collectors import collectors as cl
from collectors import meraki_collectors as mc
importlib.reload(cl)
importlib.reload(hp)
importlib.reload(rc)
importlib.reload(mc)

# Create a dataframe of collectors to run
print('COLLECTORS TO RUN\n')
df_collectors = create_collectors_df(collector_select, hostgroup_select)
display(df_collectors.style.hide(axis="index"))

# Set the timestamp so it will be consistent for all collectors
ts = dt.datetime.now()
ts = ts.strftime('%Y-%m-%d_%H%M')

# Execute the collectors
for idx, row in df_collectors.iterrows():
    ansible_os = row['ansible_os']
    hostgroup = row['hostgroup']
    collector = row['collector']
    result = rc.collect(collector,
                        nm_path,
                        private_data_dir,
                        ts,
                        ansible_os=ansible_os,
                        username=username,
                        password=password,
                        api_key=api_key,
                        hostgroup=hostgroup,
                        play_path=play_path,
                        db_path=db_path,
                        orgs=orgs,
                        networks=networks,
                        method='append')
    print(f'\nRESULT: {ansible_os.upper()} {collector.upper()} COLLECTOR\n')
    display(result)

# Run Validators

## Reload imported modules (optional; used for testing)

In [ ]:
import helpers as hp
import run_collectors as rc
import validators as vl
from collectors import collectors as cl
from collectors import meraki_collectors as mc

importlib.reload(cl)
importlib.reload(mc)
importlib.reload(hp)
importlib.reload(rc)
importlib.reload(vl)

## F5 Validators

### ```f5_vip_availability```

In [ ]:
df_diff = vl.f5_vip_availability(db_path, 'f5_vip_availability')
display(df_diff)

### ```f5_pool_availability```

In [ ]:
df_diff = vl.f5_pool_availability(db_path, 'f5_pool_availability')
display(df_diff)

### ```f5_pool_member_availability```

In [ ]:
df_diff = vl.f5_pool_member_availability(db_path, 'f5_pool_member_availability')
display(df_diff)

## Meraki Validators

### ```meraki_get_org_device_statuses```

In [ ]:
%%time
df_diff = vl.meraki_device_statuses_availability(db_path, 'meraki_get_org_device_statuses')
display(df_diff.sort_values(by='networkId', ascending=True).style.hide(axis="index"))